# Elementary Benchmark and Feature Selection 

Eventhough I started to learn about machine learning, I always struggle with the question 'where to begin a new machine learning challenge?'. A lot of good literature is available, but also blog entries etc. can give advice what has to be done in principle. However, the risk remains that as soon as I have to do it by myself, everything becomes somewhat blank. This notebook shows my very first steps I chose to approach Kaggle's House Prices competition in the getting started section.  

Three things I did when starting this getting started competition:<br/>
1) Load data and have a first glimps at it<br/>
2) a very trivial (elementary) benchmark, which will serve as a starting point for my house price prediction model<br/>
3) one possible approach for feature selection to improve the trivial benchmark from above<br/>

This notebook is inspired by ... and ...

## Load data and have a first glimps at it
set up a new project with the following structure<br>
<b>HousePrices</b> <br>
<b>|- data</b> *contains the train and test data* <br>
<b>|- output</b> *contains generated output files, such as the submission file or an overview.xlsx as explained below etc.* <br>
<b>|- src </b> *contains source files e.g. this notebook or .py files etc.*

In [50]:
# relevant imports
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [51]:
# load the train and test data
train_data = pd.read_csv('../data/train.csv', sep=',', header=0)
test_data = pd.read_csv('../data/test.csv', sep=',', header=0)

# have a look at train_data shape. It consists of 1460 observations (rows) and 81 features (columns)
print(train_data.shape)

(1460, 81)


In [52]:
# create an csv file with all the attributes in the train_data, including their data types. 
# add additional information I want to capture to each attribute. E.g. my expectation of the attributes relevance etc

attributes = list(train_data.columns.values)
attr_types = list(train_data.dtypes)

overview = pd.DataFrame({"AttributeNames": attributes, "DataType": attr_types, "VarType": "", "Expectation": "",
                         "Conclusion": "", "Comments": ""})
    
overview.to_csv("../output/overview.csv", columns=["AttributeNames", "DataType", "VarType", "Expectation", 
                                                   "Conclusion", "Comments"], header=True, index=False)

Now I started to read through each attribute. What is meant by it, do I expect that it will be relevant regarding the SalePrice, is it a categorical or a quantitative attribute etc.

As an example: From my personal experience and some research in the internet, I came to the conclusion that the area in square meters will be of high relevance, but for example the form of the Hausdach maybe of low relevance and so on. 

After a first iteration the file still had a lot of gaps in it, but I had a first impression what the attributes are in there and what they mean. 

In a next step I started to do some descriptive (multivariate) analysis to gain additional insights. 

## Missing Data

- Analyse missing data. How much is actually missed in each feature
- decide on deleting or completing the feature or on deleting the respective observations causing the missing value.
- delete or impute, as decided above.

In [53]:
# analyze missing data

total = train_data.isnull().sum().sort_values(ascending=False)
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['TotalMissing', 'Percent'])

print(missing_data.head(20))

              TotalMissing   Percent
PoolQC                1453  0.995205
MiscFeature           1406  0.963014
Alley                 1369  0.937671
Fence                 1179  0.807534
FireplaceQu            690  0.472603
LotFrontage            259  0.177397
GarageCond              81  0.055479
GarageType              81  0.055479
GarageYrBlt             81  0.055479
GarageFinish            81  0.055479
GarageQual              81  0.055479
BsmtExposure            38  0.026027
BsmtFinType2            38  0.026027
BsmtFinType1            37  0.025342
BsmtCond                37  0.025342
BsmtQual                37  0.025342
MasVnrArea               8  0.005479
MasVnrType               8  0.005479
Electrical               1  0.000685
Utilities                0  0.000000


In a first step, I decided to delete all the features (columns) that have more than 5% missing values (e.g. PoolQC, MiscFeature). Additionally, to delete all the observations (rows) that are affected by missing values. I will store this changed dataframe under a new benchmark_data name, as it is radically changing the original dataframe. I decided to go with this mixed column/row delection approach, to soften the radicality. I.e. instead of deleting 8 additional features, I only took out several observations that bring the feature onto this list (e.g. BsmtExposure, BsmtFinType2). 

Again, this is very radical and maybe not really improving a future model. 
However, I start with this approach first to be able to submit a high-level benchmark first, before going more into detail.  I will re-think this step later on and follow another strategy then. 

In [54]:
# delete features (columns) with more than 5% missing values
benchmark_data = train_data.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage',
                                 'GarageCond', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual'], axis=1)

# delete observations (rows) with a missing value in it
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['BsmtExposure'].isnull()].index)
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['BsmtFinType2'].isnull()].index)
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['BsmtFinType1'].isnull()].index)
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['BsmtCond'].isnull()].index)
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['BsmtQual'].isnull()].index)
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['MasVnrArea'].isnull()].index)
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['MasVnrType'].isnull()].index)
benchmark_data = benchmark_data.drop(benchmark_data.loc[benchmark_data['Electrical'].isnull()].index)

# how is the dataframe shaped now?
print(benchmark_data.shape)

(1412, 70)


vs. (1460, 81) in the original traindata set. Let's proceed with this benchmark_data and come back to this step later on. 

## Elementary benchmark including first submission

In [58]:
benchmark_data.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotArea            int64
Street            object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
BsmtQual          object
BsmtCond          object
                  ...   
Electrical        object
1stFlrSF           int64
2ndFlrSF           int64
LowQualFinSF       int64
GrLivArea          int64
BsmtFullBath       int64
BsmtHalfBath       int64
FullBath           int64
HalfBath           int64


In [ ]:

categories = np.union1d(train, test)
train.apply(lambda x:x.astype('category', categories=categories), axis=0)

In [27]:
# convert categorical features - which are all strings in the current dataframe - into codes
benchmark_data = pd.get_dummies(benchmark_data, drop_first=True)

# split the benchmark_data into a train and a valid set.
target_variable = benchmark_data['SalePrice']
features = benchmark_data.drop(['Id', 'SalePrice'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(features, target_variable, test_size=0.2, random_state=0)

# train a simple random forest regressor on the X_train part of the benchmark_data
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)

# make a prediction for the X_valid part of the benchmark_data, based on the trained model
y_pred = rf_reg.predict(X_valid)

# evaluate performance of your trained model on the X_valid part of the benchmark_data
loss = mean_squared_error(y_valid, y_pred)
print('Loss: ', loss)

score = rf_reg.score(X_valid, y_valid)
print('Score: ', score)

Loss:  841733202.478
Score:  0.869051001041


In [48]:
# prepare test data. Apply all relevant steps from above
test_ids = test_data.Id
test_data = test_data.drop(['Id'], axis=1)

test_data = test_data.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage',
                                 'GarageCond', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual'], axis=1)

test_data = test_data.drop(test_data.loc[test_data['BsmtExposure'].isnull()].index)
test_data = test_data.drop(test_data.loc[test_data['BsmtFinType2'].isnull()].index)
test_data = test_data.drop(test_data.loc[test_data['BsmtFinType1'].isnull()].index)
test_data = test_data.drop(test_data.loc[test_data['BsmtCond'].isnull()].index)
test_data = test_data.drop(test_data.loc[test_data['BsmtQual'].isnull()].index)
test_data = test_data.drop(test_data.loc[test_data['MasVnrArea'].isnull()].index)
test_data = test_data.drop(test_data.loc[test_data['MasVnrType'].isnull()].index)
test_data = test_data.drop(test_data.loc[test_data['Electrical'].isnull()].index)

test_data = pd.get_dummies(test_data, drop_first=True)
test_data = test_data.fillna(test_data.mean())

# apply model on test data and make predictions
predictions = rf_reg.predict(test_data)

# prepare submission
submission = pd.DataFrame({"Id": test_ids,"SalePrice": predictions})
print(submission.head(20))

ValueError: Number of features of the model must match the input. Model n_features is 211 and input n_features is 195 

In [19]:
print(predictions)

[ 107850.   208430.   181959.   187230.   163800.   206234.9  332282.8
  171293.2  123971.   172290.    92540.   180130.   216998.   144200.
  259490.4  193008.   204740.   122400.   149535.   190783.5  145410.
  146910.   219556.5  132590.   131948.   265648.   175039.9  119878.7
  221220.   149525.   212350.    82000.   152640.   151300.   169740.
  120650.   137600.   161200.   142553.   210882.8  137740.   185073.2
  341213.7  129290.   178739.   137060.   174750.    89300.   246110.
  202175.   265550.   130190.   119330.   256301.3  323311.6  192500.
  130540.   135445.   148690.   272520.   124870.   184228.4  153386.6
  296627.7  119125.   247332.8  144930.    83300.   145000.   105750.
  144095.   268730.   167385.   117990.8  157263.4  124890.   192580.
  189306.5  176042.2  218430.   142885.   170170.   315405.   153940.
  121450.   239190.   132540.9  192157.9  193887.   233000.   191435.
   91650.   265200.   147900.   216600.   180700.   151150.   193240.
  131145.   1667

In [26]:
rf_reg.score(X_valid, y_valid)

0.86251890799147835

## Feature selection

### delete features with zero variance

### delete (two) features with identical values